## Preposcessing with ES & EZ

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from datetime import date
import datetime
import pandas_datareader.data as web
import pickle
import yfinance as yf

In [2]:
# start = '2019-01-01'
# end = '2021-12-17'

# assets = ['^GSPC']

# SPY_data = yf.download(assets, start = start, end = end)
# SPY_data.to_csv("/Users/gaojinglun/Desktop/RSG/SP500.csv")

In [3]:
SP500 = pd.read_csv("SP500.csv")
SP500['Date'] = SP500['Date'].apply(lambda x : x[:4] + x[5:7] + x[-2:]).values
SP500.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,20190102,2476.959961,2519.489990,2467.469971,2510.030029,2510.030029,3733160000
1,20190103,2491.919922,2493.139893,2443.959961,2447.889893,2447.889893,3822860000
2,20190104,2474.330078,2538.070068,2474.330078,2531.939941,2531.939941,4213410000
3,20190107,2535.610107,2566.159912,2524.560059,2549.689941,2549.689941,4104710000
4,20190108,2568.110107,2579.820068,2547.560059,2574.409912,2574.409912,4083030000


In [4]:
trading_date = np.array(
    list(SP500['Date'].values) + ['20211206', '20211207', '20211208', '20211209', '20211210', '20211213', 
                                  '20211214', '20211215', '20211216', '20211217']
)
list(trading_date).index('20200102')

252

In [5]:
daily_change = (SP500['Adj Close'].values / SP500['Adj Close'].shift(1).values) - 1
daily_change[:10]

array([        nan, -0.02475673,  0.03433571,  0.00701043,  0.00969529,
        0.00409805,  0.00451842, -0.0001463 , -0.00525753,  0.01072169])

In [6]:
# 1202 in es while 1201 in ez
# Read in the data and combine them into one dataframe
es_ez_date = np.array([
    20200102, 20200115, 20200203, 20200214, 20200302, 20200316, 20200401, 20200415, 20200501, 20200515, 20200601,
    20200615, 20200701, 20200715, 20200803, 20200814, 20200901, 20200915, 20201001, 20201015, 20201102, 20201116,
    20201215, 20210104, 20210115, 20210201, 20210216, 20210301, 20210315 
])
ES_EZ = None
for i in es_ez_date:
    # Read the es data
    es_path = '/Users/gaojinglun/Desktop/eod/es/' + str(i) + '.csv'
    ES_small = pd.read_csv(es_path, usecols = ['Trade.Date', 'Contract.Year', 
                                               'Contract.Month', 'Settlement', 'Last.Trade.Date'])
    
    # Read the ez data
    ez_path = '/Users/gaojinglun/Desktop/eod/ez/' + str(i) + '.csv'
    EZ_small = pd.read_csv(ez_path, usecols = ['Trade.Date', 'Put.Call', 'Strike.Price', 'Contract.Year', 
                                               'Contract.Month', 'Settlement', 'Open.Interest',
                                               'Delta', 'Implied.Volatility', 'Last.Trade.Date'])
    
    futures_price = np.zeros(EZ_small.shape[0])
    for i in range(EZ_small.shape[0]):
        if EZ_small['Contract.Month'][i] == 3:
            futures_price[i] = ES_small['Settlement'][0]
        elif EZ_small['Contract.Month'][i] == 6:
            futures_price[i] = ES_small['Settlement'][1]
        elif EZ_small['Contract.Month'][i] == 9:
            futures_price[i] = ES_small['Settlement'][2]
        elif EZ_small['Contract.Month'][i] == 12:
            futures_price[i] = ES_small['Settlement'][3]
            
    assert np.sum(futures_price == 0) == 0, 'futures price should be positive!'
    
    EZ_small['futures.price'] = futures_price
    ES_EZ = pd.concat([ES_EZ, EZ_small], axis = 0)
    
    del ES_small    
    del EZ_small

In [7]:
print('There are {} rows and {} columns in ES_EZ data'.format(ES_EZ.shape[0], ES_EZ.shape[1]))

There are 29971 rows and 11 columns in ES_EZ data


In [8]:
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0


In [9]:
# Calculate the time to maturity
T = np.zeros(ES_EZ.shape[0])
for i in range(len(T)):
    dateInString = str(ES_EZ['Trade.Date'].values[i])
    start_year = int(dateInString[:4])
    
    if dateInString[4] == str(0):
        start_month = int(dateInString[5])
    else:
        start_month = int(dateInString[4:6])
    
    if dateInString[6] == str(0):
        start_day = int(dateInString[-1])
    else:
        start_day = int(dateInString[6:])
    
    # Save the start date in date
    start_date = date(start_year, start_month, start_day)
    
    endDateInString = str(ES_EZ['Last.Trade.Date'].values[i])
    end_year = int(endDateInString[:4])
    
    if endDateInString[4] == str(0):
        end_month = int(endDateInString[5])
    else:
        end_month = int(endDateInString[4:6])
    
    if endDateInString[6] == str(0):
        end_day = int(endDateInString[-1])
    else:
        end_day = int(endDateInString[6:])
    
    # Save the start date in date
    end_date = date(end_year, end_month, end_day)
    
    T[i] = (end_date - start_date).days
    
assert np.sum(T == 0) == 0, 'Time should be positive!'

In [10]:
ES_EZ['Time.to.maturity'] = T 
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price,Time.to.maturity
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0,78.0
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0,78.0
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0,78.0
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0,78.0
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0,78.0


In [11]:
def Calculate_trading_days(Start, End):
    '''
    Calculate the number of trading days remaining
    '''
    start_idx = list(trading_date).index(str(Start))
    End_idx = list(trading_date).index(str(End))
    return End_idx - start_idx

In [12]:
num_trading_days = np.zeros(ES_EZ.shape[0])
for i in range(ES_EZ.shape[0]):
    num_trading_days[i] = Calculate_trading_days(ES_EZ['Trade.Date'].values[i], ES_EZ['Last.Trade.Date'].values[i])

In [13]:
hist_vol = np.zeros(ES_EZ.shape[0])
for i in range(ES_EZ.shape[0]):
    trade_date_idx = list(SP500['Date'].values).index(str(ES_EZ['Trade.Date'].values[i]))
    annualization_factor = np.sqrt(12 / ES_EZ['Contract.Month'].values[i])
    hist_vol[i] = np.std(daily_change[int(trade_date_idx - num_trading_days[i]) : trade_date_idx])
    hist_vol[i] *= annualization_factor

In [14]:
ES_EZ['Historical Vol'] = hist_vol
ES_EZ['Time.to.maturity'] = ES_EZ['Time.to.maturity'].values / 365
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price,Time.to.maturity,Historical Vol
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0,0.213699,0.008335
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0,0.213699,0.008335
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0,0.213699,0.008335
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0,0.213699,0.008335
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0,0.213699,0.008335


In [15]:
# Add the risk-free-rate
# T-bill
T_bill_rate = np.zeros(len(T))

def t_bill_distance(x):
    '''
    Find the similar type of the T-bill based on the time to maturity
    '''
    one_month_t_bill = int(365 / 12)
    three_month_t_bill = int(365 / 4)
    six_month_t_bill = int(365 / 2)
    one_year_t_bill = int(365 / 1)
    idx = np.argmin([
        np.abs(x - one_month_t_bill), np.abs(x - three_month_t_bill), 
        np.abs(x - six_month_t_bill), np.abs(x - one_year_t_bill)
    ])
    return [0, 1, 2, 3][idx]

for i in range(len(T)):
    T_bill_rate[i] = t_bill_distance(T[i])
np.unique(T_bill_rate)

array([0., 1., 2., 3.])

In [16]:
TB_data = np.array(['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1'])
TB = []
TB_date_new = []
for i in range(4):
    TB_path = '/Users/gaojinglun/Desktop/eod/' + TB_data[i] + '.csv'
    TB_small = pd.read_csv(TB_path)
    
    def DATE_convertor(x):
        '''
        convert xxxx-xx-xx to xxxxxxxx
        '''
        return x[:4] + x[5:7] + x[-2:]

    date_idx = np.zeros(TB_small.shape[0], dtype = bool)
    for i in range(TB_small.shape[0]):
        new_date = int(DATE_convertor(TB_small['DATE'].values[i]))
        if new_date in es_ez_date:
            date_idx[i] = True
            TB_date_new.append(new_date)
    
    TB.append(TB_small[TB_small.columns.values[-1]][date_idx].values)
    del TB_small
    
TB = np.array(TB)
TB_date_new = TB_date_new[:int(len(TB_date_new) / 4)]
TB.shape

(4, 29)

In [17]:
risk_free_rate = np.zeros(ES_EZ.shape[0])
for i in range(ES_EZ.shape[0]):
    risk_free_rate[i] = TB[int(T_bill_rate[i]), TB_date_new.index(ES_EZ['Trade.Date'].values[i])]

In [18]:
ES_EZ['Risk.Free.Rate'] = risk_free_rate / 100
ES_EZ.head()

,Trade.Date,Put.Call,Strike.Price,Contract.Year,Contract.Month,Settlement,Open.Interest,Delta,Implied.Volatility,Last.Trade.Date,futures.price,Time.to.maturity,Historical Vol,Risk.Free.Rate
0,20200102,C,2200,2020,3,1059.0,826.0,0.99787,0.338267,20200320,3259.0,0.213699,0.008335,0.0154
1,20200102,C,2250,2020,3,1009.1,728.0,0.99661,0.326829,20200320,3259.0,0.213699,0.008335,0.0154
2,20200102,C,2270,2020,3,989.1,1.0,0.99586,0.323456,20200320,3259.0,0.213699,0.008335,0.0154
3,20200102,C,2300,2020,3,959.2,212.0,0.99474,0.317834,20200320,3259.0,0.213699,0.008335,0.0154
4,20200102,C,2310,2020,3,949.2,42.0,0.99466,0.314282,20200320,3259.0,0.213699,0.008335,0.0154


In [19]:
ES_EZ.to_csv("/Users/gaojinglun/Desktop/RSG/ES_EZ.csv")

**Reference**

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 1-Month Constant Maturity [DGS1MO], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS1MO, November 7, 2021.

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 3-Month Constant Maturity [DGS3MO], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS3MO, November 7, 2021.

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 6-Month Constant Maturity [DGS6MO], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS6MO, November 7, 2021.

- Board of Governors of the Federal Reserve System (US), Market Yield on U.S. Treasury Securities at 1-Year Constant Maturity [DGS1], retrieved from FRED, Federal Reserve Bank of St. Louis; https://fred.stlouisfed.org/series/DGS1, November 7, 2021.